In [1]:
'''
This notebook uses Long Short-Term Memory (LSTM) on multivariate time series data to predict the closing 
stock price of a corporation using the past 60 day stock movement.
'''
# Import the libraries
import math
import pandas_datareader as web
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.autograd import Variable
plt.style.use('fivethirtyeight')

# Get cuda device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

/home/victoru/anaconda3/envs/testpython/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Get the past stock price
# today's date
today = pd.to_datetime('today').strftime('%Y-%m-%d')
# 5 year ago
start = pd.to_datetime('today') - pd.DateOffset(years=5)
df = web.DataReader('AAPL', data_source='yahoo', start=start, end=today)

In [3]:
# prepare data
target_data = "Close"
feature = list(df.columns.difference([target_data]))

forecast_lead = 1
target = f"{target_data}_t+{forecast_lead}"

df[target] = df[target_data].shift(-forecast_lead)
df = df.iloc[:-forecast_lead]

# split data
train_size = int(len(df) * 0.8)
test_size = len(df) - train_size
train, test = df.iloc[0:train_size].copy(), df.iloc[train_size:len(df)].copy()

In [4]:
# standardize data
target_mean = train[target].mean()
target_std = train[target].std()

for c in train.columns:
    mean = train[c].mean()
    std = train[c].std()

    train[c] = (train[c] - mean) / std
    test[c] = (test[c] - mean) / std

In [5]:
# define custom dataset
class SequenceDataset(torch.utils.data.Dataset):
    def __init__(self, dataframe, target, features, seq_len = 60):
        self.dataframe = dataframe
        self.target = target
        self.seq_len = seq_len
        self.x = torch.tensor(dataframe[features].values, dtype=torch.float32)
        self.y = torch.tensor(dataframe[target].values, dtype=torch.float32)

    def __len__(self):
        return self.x.shape[0]

    def __getitem__(self, idx):
        if idx >= self.seq_len - 1:
            idx_start = idx - self.seq_len + 1
            x = self.x[idx_start:idx+1,:]
        else:
            padding = self.x[0].repeat(self.seq_len - idx - 1, 1)
            x = self.x[0:(idx+1),:]
            x = torch.cat((padding, x), dim=0)

        return x, self.y[idx]

In [6]:
# create dataset and data loader
batch_size = 4
seq_len = 60

torch.manual_seed(seq_len*batch_size/2)

train_dataset = SequenceDataset(train, target, feature, seq_len)
test_dataset = SequenceDataset(test, target, feature, seq_len)
train_looader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# get the first batch and its shape
x, y = next(iter(train_looader))
print("Features shape: ", x.shape)
print("Target shape: ", y.shape)

Features shape:  torch.Size([4, 60, 5])
Target shape:  torch.Size([4])


In [7]:
# define model
class ShallowRegressionLSTM(nn.Module):
    def __init__(self,n_features, n_hidden, n_layers=1):
        super(ShallowRegressionLSTM, self).__init__()
        self.n_hidden = n_hidden
        self.n_layers = n_layers

        self.lstm = nn.LSTM(
            input_size = n_features, 
            hidden_size=n_hidden, 
            num_layers=n_layers, 
            batch_first=True)
        self.linear = nn.Linear(in_features = self.n_hidden, out_features=1)
    
    def forward(self, x):
        batch_size = x.size(0)
        h0 = torch.zeros(self.n_layers, batch_size, self.n_hidden).requires_grad_().to(device)
        c0 = torch.zeros(self.n_layers, batch_size, self.n_hidden).requires_grad_().to(device)
        _, (hn, _) = self.lstm(x, (h0, c0))
        out = self.linear(hn[0]).flatten() #first dim of hn is the layer dimension
        return out

In [8]:
# create model, loss and optimizer
learning_rate = 5e-5
num_hidden = 16

model = ShallowRegressionLSTM(n_features = len(feature), n_hidden = num_hidden).to(device)
loss = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [9]:
# define training function and testing function
def train(model, loader, loss, optimizer, device):
    model.train()
    train_loss = 0
    for x, y in loader:
        x = x.to(device)
        y = y.to(device)
        optimizer.zero_grad()
        y_pred = model(x)
        batch_loss = loss(y_pred, y)
        batch_loss.backward()
        optimizer.step()
        train_loss += batch_loss.item()

    avg_loss = train_loss / len(loader)
    print(f"Train loss: {avg_loss:.4f}")
    
def test(model, loader, loss, device):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)
            y_pred = model(x)
            batch_loss = loss(y_pred, y)
            test_loss += batch_loss.item()

    avg_loss = test_loss / len(loader)
    print(f"Test loss: {avg_loss:.4f}")

In [10]:
# train and test model
print("Untrained test\n----------------")
test(model, test_loader, loss, device)
print("\nTraining\n----------------")

for epoch in range(10):
    print(f"Epoch {epoch+1}\n")
    train(model, train_looader, loss, optimizer, device)
    test(model, test_loader, loss, device)

Untrained test
----------------
Test loss: 6.5109

Training
----------------
Epoch 1

Train loss: 0.9636
Test loss: 5.6321
Epoch 2

Train loss: 0.8094
Test loss: 4.7004
Epoch 3

Train loss: 0.6256
Test loss: 3.6037
Epoch 4

Train loss: 0.4267
Test loss: 2.6357
Epoch 5

Train loss: 0.2682
Test loss: 1.9482
Epoch 6

Train loss: 0.1660
Test loss: 1.5050
Epoch 7

Train loss: 0.1046
Test loss: 1.2088
Epoch 8

Train loss: 0.0712
Test loss: 1.0095
Epoch 9

Train loss: 0.0518
Test loss: 0.8719
Epoch 10

Train loss: 0.0422
Test loss: 0.7728
